# Workspace preparation

In [1]:
from azure.ai.ml import command, Input, MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute, Environment, Data, AzureBlobDatastore, AccountKeyConfiguration
from azure.ai.ml.constants import AssetTypes
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, generate_blob_sas, BlobSasPermissions

from dotenv import load_dotenv

import os

In [2]:
load_dotenv()

SUBSCRIPTION_ID = os.environ.get("SUBSCRIPTION_ID")
RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP")
WORKSPACE_NAME = os.environ.get("WORKSPACE_NAME")
DATASTORE_NAME = os.environ.get("DATASTORE_NAME")
ACCOUNT_NAME = os.environ.get("ACCOUNT_NAME")
CONTAINER_NAME = os.environ.get("CONTAINER_NAME")
COMPUTE_CLUSTER_NAME = os.environ.get("COMPUTE_NAME")
LOCATION = os.environ.get("LOCATION")
CONNECTION_KEY = os.environ.get("CONNECTION_KEY")
ACCOUNT_KEY= os.environ.get("ACCOUNT_KEY")
DATASET_NAME= os.environ.get("DATASET_NAME")

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(),
    SUBSCRIPTION_ID,
    RESOURCE_GROUP,
    WORKSPACE_NAME
)

In [4]:
# create compute cluster
cluster_basic = AmlCompute(
    name=COMPUTE_CLUSTER_NAME, 
    type="aml-compute",
    size="STANDARD_D4s_v3",
    location=LOCATION,
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=120,
    tier="dedicated"
)

ml_client.begin_create_or_update(cluster_basic)

In [5]:
# create environment if it doesn't exist
dependencies_dir = "./deps"
custom_job_env = Environment(
    name="keras-env",
    description="Environment for Tensorflow Experiments",
    tags={"keras" : "3.1.1"},
    conda_file=os.path.join(dependencies_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu20.04",
    version="v5"
)

ml_client.create_or_update(custom_job_env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'keras-env', 'description': 'Environment for Tensorflow Experiments', 'tags': {'keras': '3.1.1'}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/8af766d1-2a08-48cd-8f6e-71c0ecdf262f/resourceGroups/prof-cs-rg/providers/Microsoft.MachineLearningServices/workspaces/worksp-proj-cs/environments/keras-env/versions/v5', 'Resource__source_path': '', 'base_path': 'c:\\Users\\j.martins\\Documents\\AI_HUB_Academy_training\\Capstone_project\\review-analysis-teamc\\code\\aml', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000001CE1F3EF550>, 'serialize': <msrest.serialization.Serializer object at 0x000001CE1F3EE140>, 'version': 'v5', 'conda_file': {'channe

In [6]:
# create container if it doesn't exist
try:
    blob_client = BlobServiceClient.from_connection_string(CONNECTION_KEY)
    container_client = blob_client.create_container(name=CONTAINER_NAME)
except Exception as e:
    print(e)

In [7]:
# upload reviews dataset to container
try:
    blob_client = BlobServiceClient.from_connection_string(CONNECTION_KEY)
    container_client = blob_client.get_container_client(container=CONTAINER_NAME)
    with open(f"../../data/{DATASET_NAME}", "rb") as data: # here replace with the location to your data
        container_client.upload_blob(name=DATASET_NAME, data=data, overwrite=True)
except Exception as e:
    print(e)

In [12]:
CONTAINER_NAME

'cont-proj-cs'

In [ ]:
credentials=AccountKeyConfiguration(account_key=ACCOUNT_KEY)
 
store = AzureBlobDatastore(
    name=DATASTORE_NAME,
    description="Amazon Reviews datastore",
    account_name=ACCOUNT_NAME,
    container_name=CONTAINER_NAME,
    credentials=credentials
)
 
ml_client.create_or_update(store)

In [11]:
store = ml_client.data.get(name=DATASTORE_NAME, version="1")

ResourceNotFoundError: (UserError) Data version datastore-proj-cs:1 (dataContainerName:version) not found.
Code: UserError
Message: Data version datastore-proj-cs:1 (dataContainerName:version) not found.

In [ ]:
# create Data Asset from Datastore
datastore_uri = f"azureml://subscriptions/{SUBSCRIPTION_ID}/resourcegroups/{RESOURCE_GROUP}/workspaces/{WORKSPACE_NAME}/datastores/{store.name}/paths/"

amazon_folder = Data(
    path=datastore_uri,
    type=AssetTypes.URI_FOLDER,
    description="amazon reviews folder",
    name="amazon_reviews_folder",
    version="1"
)

try:
    data_asset = ml_client.data.get(name="amazon_reviews_folder", version="1")
    print(
        f"Data asset already exists. Name: {data_asset.name}, version: {data_asset.version}"
    )
except Exception as e:
    print(e)
    ml_client.data.create_or_update(amazon_folder)
    print(f"Data asset created. Name: {amazon_folder.name}, version: {amazon_folder.version}")